### S3로 CSV파일 업로드

In [4]:
PROJECT = "emart-datafabric"
DATASET = "common_dev"
TABLE = "dfm_sample_eapp_review_keywords"
LIMIT = 30

In [5]:
query = f"select * from {PROJECT}.{DATASET}.{TABLE}"
path = "app_review_keywords/csv"

In [8]:
import os
from pydatafabric.vault_utils import get_secrets

aws_info = get_secrets(mount_point="datafabric",path="aws/credentials/datafabric")

In [9]:
env = "dev" 
bucket = f"emart-datafabric-{env}"
destination = f"s3a://{bucket}/bigquery-db/{path}/op=put/"

In [10]:
from pydatafabric.ye import get_spark

spark = get_spark()
spark.conf.set("fs.s3a.fast.upload.buffer", "disk")
spark.conf.set("fs.s3a.buffer.dir", "/tmp")
spark.conf.set("fs.s3a.access.key", aws_info["aws_access_key_id"])
spark.conf.set("fs.s3a.secret.key", aws_info["aws_secret_access_key"])

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/04 15:29:57 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
22/10/04 15:29:57 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
22/10/04 15:29:57 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
22/10/04 15:29:57 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


### CSV 포맷으로 저장

In [11]:
from pydatafabric.gcp import bq_to_df

df = bq_to_df(query, spark_session=spark)
df.write.mode("overwrite").csv(destination)

22/10/04 15:30:16 WARN org.apache.hadoop.fs.s3a.commit.AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [12]:
!hdfs dfs -Dhadoop.security.credential.provider.path=jceks:///datafabric/credentials/aws.jceks -ls {destination}

2022-10-04 15:30:30,103 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-10-04 15:30:30,155 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-10-04 15:30:30,156 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
Found 2 items
-rw-rw-rw-   1 datafabric datafabric          0 2022-10-04 15:30 s3a://emart-datafabric-dev/bigquery-db/app_review_keywords/csv/op=put/_SUCCESS
-rw-rw-rw-   1 datafabric datafabric   49727095 2022-10-04 15:30 s3a://emart-datafabric-dev/bigquery-db/app_review_keywords/csv/op=put/part-00000-9e8ecd66-ae0f-45d6-8022-2396f4f4f4d8-c000.csv
2022-10-04 15:30:31,815 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-10-04 15:30:31,815 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-10-04 15:30:31,815 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.
